# Final code

In [3]:
!pip install langchain_google_genai langchain_ollama python-dotenv pandas

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached pandas-2.2.3-cp312-cp312-manylinux2014_aarch64.manylinux_2_17_aarch64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux2014_aarch64.manylinux_2_17_aarch64.whl (15.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.0 MB/s eta 0:00:0000:0100:01


In [4]:
# імпорти
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing_extensions import Literal
import pandas as pd
import time
import os
from langchain_ollama.llms import OllamaLLM
from dotenv import load_dotenv

In [2]:
# Завантажуємо .env файл
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
# Use Gemini
model_name = "gemini-2.0-flash"
llm = ChatGoogleGenerativeAI(
    model=model_name,
    api_key=api_key,
    temperature=0.8,
    max_retries=2,
    top_p=0.9,
    top_k=40,
)

# Use local model
# llm = OllamaLLM(
#     base_url=os.environ["BASE_URL"],
#     model="aya-expanse:32b-q2_K",
#     keep_alive="3h",
#     temperature=0.7,
#     format="json",
#     top_p=0.9
# )

In [4]:
# Ініціалізація моделі
class OutputModel(BaseModel):
    title: str = Field(description="Заголовок новини")
    text: str = Field(description="Текст новини (300-500 слів)")
    # topic: str = Field(description="Обрана тема новини")
    # type_news: str = Field(description="Обраний тип новини")

parser = JsonOutputParser(pydantic_object=OutputModel)

In [5]:
# System message
system_message = """Ви — досвідчений журналіст із понад 10 роками роботи в провідних українських та міжнародних медіа.
Ваші тексти вирізняються глибиною аналізу, структурованістю та дотриманням високих стандартів журналістики.
Ви володієте майстерністю фактчекінгу, балансу думок і доступного викладу складних тем.\
"""

# Human message
human_message_template = """Згенеруй **фейкову, але правдоподібну** новину у стилі провідних українських медіа 2022–2025 років. \

### **Контекст подачі:**
- Тональність: {tone}
- Стиль: {style}
- Тип: {type_of_news}
- Тематика: {topics}

---

### Вимоги до тексту:
- **Почни з головного** — подія, її суть, причетні особи або інституції.
- Пиши **доступною, живою мовою** — без офіціозу, штампів чи технократичного жаргону.
- Стиль має бути **максимально реалістичним**, у форматі репортажу або аналітичної замітки.
- Включай **реальні імена посадовців, державних установ і фактів** (у межах 2022–2025 рр.).
- Додай **коментарі експертів або посадових осіб** — вигадані, але переконливі.
- **Не вигадуй географічних назв**, не використовуй очевидні кліше чи фантастичні події.
- Орієнтуйся на стиль **«Української правди», «bihus.info», «ТСН»** — стриманий, динамічний, фактологічний.

---

# ВИВІД МАЄ БУТИ У ФОРМАТІ:
{format_of_output}
"""

In [6]:
import random
from langchain_core.prompts import ChatPromptTemplate

tones = ["Нейтральна", "Тривожна", "Заспокійлива"]
styles = ["Суха подача", "Драматична", "Аналітична", "Іронічна", "Популістська", "Фактологічна"]
type_of_news = ["Маніпуляція", "Дезінформація", "Пропаганда", "Вплив через емоції",]
topics = ["Політика", "Економіка", "Суспільство", "Військові події", "Освіта", "Місцеве самоврядування", "Зовнішня політика", "Енергетика", "Інфраструктура", "Охорона здоров'я", "Культура", "Довкілля"]


# Генерація новини
def generate_one_news():
    prompt = ChatPromptTemplate(
        [
            ("system", system_message),
            ("human", human_message_template)
        ]
    )

    chain = prompt | llm | parser

    # Рандомно вибираємо тон, стиль, тип новин
    tone = random.choice(tones)
    style = random.choice(styles)
    type = random.choice(type_of_news)
    topic = random.choice(topics)

    input_data = {
        "tone": tone,
        "style": style,
        "type_of_news": type,
        "topics": topic,
        "format_of_output": parser.get_format_instructions()
    }

    # Запускаємо модель
    if isinstance(llm, OllamaLLM):
        return chain.invoke(input_data)['properties']
    else:
        return chain.invoke(input_data)

In [7]:
result = generate_one_news()
result

{'title': 'Угорщина блокує €500 млн військової допомоги Україні, вимагаючи гарантій для угорської меншини на Закарпатті – джерела',
 'text': "Будапешт знову ставить палки в колеса європейській підтримці України. За інформацією джерел в дипломатичних колах ЄС, Угорщина заблокувала черговий транш у розмірі 500 мільйонів євро з Європейського фонду миру, призначеного для військової допомоги Україні. Причина – нібито недостатній захист прав угорської меншини в Закарпатській області.\n\n«Угорська сторона наполягає на наданні Києвом письмових гарантій щодо дотримання прав угорців в Україні, зокрема, у сфері освіти та мови. Без цього, за словами угорських дипломатів, вони не готові підтримати подальші транші військової допомоги», – повідомив співрозмовник УП в Брюсселі.\n\nВарто нагадати, що це не перший випадок, коли Угорщина блокує рішення, важливі для України, прикриваючись захистом прав меншин. У 2023 році Будапешт неодноразово затягував процес узгодження санкцій проти Росії, вимагаючи вик

In [8]:
# додавання до датасету
def append_to_csv(news, path= "..\\..\\data\\fake_news_dataset.csv"):
    df = pd.DataFrame(
        [
            {
                "title": news['title'],
                "text": news['text'],
                "topic": news['topic'],
                "type": news['type_news']
            }
        ]
    )

    if not os.path.exists(path):
        df.to_csv(path, index=False, encoding="utf-8")
    else:
        df.to_csv(path, mode="a", index=False, header=False, encoding="utf-8")

In [ ]:
n = 1_500
delay = 15

for i in range(n):
    news = generate_one_news()

    print(f"[{i+1}/{n}] {news['title']}")

    append_to_csv(news)
    time.sleep(delay)

print(f"\n Згенеровано {n} новин. Результати збережено.")

[1/5] Міністр оборони підозрюється у корупції: хто замінить генерала Тарана?
[2/5] Сенсаційна заява міністра оборони: 'Україна готова до можливого вторгнення з боку РФ'
[3/5] Дезінформація про виведення військ РФ з Херсона спричинила паніку серед населення
[4/5] Дезінформація: ЗМІ розкривають фейкову операцію з дискредитації українського уряду
[5/5] Україна посилює оборону: нова військова стратегія

 Згенеровано 5 новин. Результати збережено.


In [53]:
import pandas as pd

df = pd.read_csv("..\\..\\data\\fake_news_dataset.csv")
df

,title,text,topic,type
0,ЗМІ: Європейські партнери висловлюють стурбова...,"Київ, Україна – Останніми тижнями зростає зане...",Політика,Маніпуляція
1,ЗМІ: Відставка Резнікова стала результатом нез...,"Київ, Україна – За інформацією, отриманою з кі...",Політика,Маніпуляція
2,Злив даних з Офісу Президента: чия вина та які...,Несподіваний витік внутрішньої документації з ...,Політика,Маніпуляція
3,Зростання експорту зерна через Польщу: чи не х...,Київ – Останні місяці демонструють стрімке зро...,Економіка,Маніпуляція
4,Зростання імпорту російського газу через посер...,"Київ, Україна – Останні місяці спостерігається...",Економіка,Маніпуляція
5,Міністр оборони підозрюється у корупції: хто з...,"У середу, 2023 року, українські ЗМІ вибухнули ...",Політика,Маніпуляція
6,Сенсаційна заява міністра оборони: 'Україна го...,"Міністр оборони України, Олексій Громов, зроби...",Політика,Дезінформація
7,Дезінформація про виведення військ РФ з Херсон...,У соціальних мережах та месенджерах поширилася...,Політика,Дезінформація
8,Дезінформація: ЗМІ розкривають фейкову операці...,"У п'ятницю, 20 березня 2025 року, кілька прові...",Політика,Дезінформація
9,Україна посилює оборону: нова військова стратегія,Уряд України оголосив про амбітний план модерн...,"Політика, Військові події",Пропаганда


In [56]:
print(df['text'][6])

Міністр оборони України, Олексій Громов, зробив несподівану заяву під час прес-конференції, яка шокувала як експертне середовище, так і громадськість. Він стверджує, що Україна повністю готова до можливого вторгнення з боку Російської Федерації, і що збройні сили країни пройшли через інтенсивні навчання та модернізацію за останні роки.

'Ми не боїмося загрози з боку Росії. Наші війська повністю готові до будь-яких дій, які може вжити Кремль. Ми провели масштабні військові навчання, оновили наш арсенал і зміцнили нашу обороноздатність', - цитує міністра офіційний сайт Міністерства оборони.

Ця заява суперечить попереднім оцінкам аналітиків, які попереджали про потенційну вразливість України перед обличчям російської агресії. Експерти з безпеки, такі як Олена Білан, висловлюють скептицизм щодо тверджень міністра:

'Хоча посилення військових можливостей України безперечно, заяви пана Громова можуть бути перебільшеними. Російська армія все ще має значну перевагу в чисельності та техніці. В